In [56]:
!pip install bertopic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
# few of the imports are just for checking while coding not included in the rest of notebook.

# Most basic stuff for EDA.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Core packages for text processing.
import string
import re

# Libraries for text preprocessing.
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords


# Loading some sklearn packaces for modelling.
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import LatentDirichletAllocation, NMF # not actively using
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Utility
import logging
import itertools


# Core packages for general use throughout the notebook.
import random
import warnings
import time
import datetime

# For customizing our plots.
from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

# for build our model
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel

# Setting some options for general use.
import os
stop = set(stopwords.words('english'))
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.5)
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


2024-03-25 16:18:30.566825: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 16:18:30.566932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 16:18:30.707800: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Read the data
df = pd.read_excel(r'C:\Users\USER\Downloads\train_sentiments.xlsx')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,2020-02-03 00:00:00,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",2020-02-03 00:00:00,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,2020-02-03 00:00:00,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,2020-02-03 00:00:00,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",2020-03-03 00:00:00,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [14]:
df.isna().sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [10]:
df['Sentiment'].unique()

array(['Extremely Negative', 'Positive', 'Extremely Positive', 'Negative',
       'Neutral'], dtype=object)

In [11]:
df.Sentiment.value_counts()

Sentiment
Negative              425
Positive              352
Extremely Positive    241
Neutral               238
Extremely Negative    213
Name: count, dtype: int64

In [16]:
df.Location.mode()[0]

'United States'

In [13]:
df['Location'].fillna(df['Location'].mode()[0], inplace=True)

In [3]:
import re
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # removing @ mention
    text = re.sub(r'#', '', text)  # removing # symbol
    text = re.sub(r'RT[\s]+', '', text)  # removing RT followed by space
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

df['OriginalTweet'] = df['OriginalTweet'].apply(cleanTxt)  # applying function
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,2020-02-03 00:00:00,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",2020-02-03 00:00:00,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,2020-02-03 00:00:00,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,2020-02-03 00:00:00,Panic buying hits NewYork City as anxious shop...,Negative
4,5,44957,"Melbourne, Victoria",2020-03-03 00:00:00,toiletpaper dunnypaper coronavirus coronavirus...,Neutral
...,...,...,...,...,...,...
1464,1465,46417,"iPhone: 41.376335,2.18889",13-03-2020,Just been to the supermarket. Why do all women...,Neutral
1465,1466,46418,"Sydney, New South Wales",13-03-2020,"In light of the pandemic, i have moved to onli...",Extremely Positive
1466,1467,46419,New Hampshire,13-03-2020,Or buy gift certificates for others who may be...,Extremely Positive
1467,1468,46420,New Hampshire,13-03-2020,"If you're going to self-quarantine, I highly r...",Extremely Positive


In [4]:
df_copy = df.copy()

In [5]:
# Importing required libraries
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
from tqdm import tqdm

# Define some key variables
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 15
LEARNING_RATE = 1e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = AutoTokenizer.from_pretrained('roberta-base')
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest',ignore_mismatched_sizes=True)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.sentiment_map = {'Extremely Negative': 0, 'Positive': 1, 'Extremely Positive': 2, 'Negative': 3, 'Neutral': 4}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        comment_text = str(self.data.loc[index, 'OriginalTweet'])
        sentiment_label = self.data.loc[index, 'Sentiment']
        target = self.sentiment_map[sentiment_label]

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(inputs['token_type_ids'], dtype=torch.long),
            'targets': torch.tensor(target, dtype=torch.long)
        }

# Split the data into train and validation sets
train_data, test_data = train_test_split(df_copy, test_size=0.3, random_state=10)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Create datasets and dataloaders
train_dataset = CustomDataset(train_data, tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_data, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, shuffle=True)

# Initialize the model
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
# model = DistilBertModel.from_pretrained("distilbert-base-uncased", num_labels=5)
# model = BertForSequenceClassification.from_pretrained('bardsai/twitter-sentiment-pl-base', num_labels=5,ignore_mismatched_sizes=True)
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=5,ignore_mismatched_sizes=True)
model.to(device)

# Define optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay = 0.00001)
total_steps = len(train_loader) * EPOCHS
scheduler = OneCycleLR(optimizer, max_lr=LEARNING_RATE, total_steps=total_steps)

# Define training function
def train_model(model, optimizer, scheduler, train_loader, test_loader, device):
    best_accuracy = 0.0  # Initialize variable to track best accuracy
    best_epoch = 0  # Initialize variable to track epoch of best accuracy
    best_model_state = None  # Initialize variable to store state of the best model
    
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0.0
        correct_train_predictions = 0
        total_train_samples = 0
        pbar = tqdm(train_loader, leave=False, total=len(train_loader))
        for batch in pbar:
            input_ids = batch['ids'].to(device)
            attention_mask = batch['mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['targets'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            logits = outputs.logits
            
            # Use the predefined criterion for computing the loss
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate

            total_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            correct_train_predictions += (predicted == labels).sum().item()
            total_train_samples += labels.size(0)
            pbar.set_description(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {loss.item():.4f}, LR: {scheduler.get_last_lr()}")

        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = correct_train_predictions / total_train_samples
        print(f"Epoch {epoch + 1}/{EPOCHS}, Average Training Loss: {avg_train_loss}, Accuracy: {train_accuracy}")

        # Evaluate the model after each epoch
        outputs, targets = evaluate_model(model, test_loader, device)
        roc_auc = roc_auc_score(targets, outputs, multi_class='ovr')
        accuracy = np.mean(np.argmax(outputs, axis=1) == targets)
        print(f"Epoch {epoch + 1}/{EPOCHS}, ROC AUC Score: {roc_auc}, Validation Accuracy: {accuracy}")

        # Save the model if it achieves the best accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_epoch = epoch + 1
            best_model_state = model.state_dict()

        # Save the best model
        if best_model_state is not None:
            torch.save(best_model_state, r'C:\Users\USER\Downloads\best_roberta_sentiment_model.pth')
            print(f"Best model saved. accuracy: {best_accuracy} at epoch {best_epoch}")

# Define function to evaluate model
def evaluate_model(model, test_loader, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['ids'].to(device)
            attention_mask = batch['mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            logits = outputs.logits
            predictions = torch.softmax(logits, dim=1)

            fin_targets.extend(labels.cpu().detach().numpy().tolist())
            fin_outputs.extend(predictions.cpu().detach().numpy().tolist())

    return np.array(fin_outputs), np.array(fin_targets)

# Train the model
train_model(model, optimizer, scheduler, train_loader, test_loader, device)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

Epoch 11/15, Average Training Loss: 0.10933729305921137, Accuracy: 0.9727626459143969
Epoch 11/15, ROC AUC Score: 0.8206890778039458, Validation Accuracy: 0.54421768707483
Best model saved. accuracy: 0.54421768707483 at epoch 11


Epoch 12/15, Average Training Loss: 0.10234626089475414, Accuracy: 0.97568093385214
Epoch 12/15, ROC AUC Score: 0.8179309507226871, Validation Accuracy: 0.5260770975056689
Best model saved. accuracy: 0.54421768707483 at epoch 11


Epoch 13/15, Average Training Loss: 0.06371181250311607, Accuracy: 0.9902723735408561
Epoch 13/15, ROC AUC Score: 0.8185654307722554, Validation Accuracy: 0.5260770975056689
Best model saved. accuracy: 0.54421768707483 at epoch 11


Epoch 14/15, Average Training Loss: 0.06694964762748212, Accuracy: 0.9863813229571985
Epoch 14/15, ROC AUC Score: 0.817809715270554, Validation Accuracy: 0.5306122448979592
Best model saved. accuracy: 0.54421768707483 at epoch 11


Epoch 15/15, Average Training Loss: 0.05867543594204178, Accuracy: 0.9902723735408561
Epoch 15/15, ROC AUC Score: 0.817665417082158, Validation Accuracy: 0.5306122448979592
Best model saved. accuracy: 0.54421768707483 at epoch 11


In [21]:
df_inference = pd.read_excel(r'C:\Users\USER\Downloads\val_sentiments.xlsx')  # Change the file name accordingly
df_inference.rename(columns={'Unnamed: 4':'OriginalTweet'},inplace=True)

In [24]:
data_for_inference = df_inference.copy()
data_for_inference

,UserName,ScreenName,Location,TweetAt,OriginalTweet
0,1470,46422,"Ontario, Canada",13-03-2020,Hey idiots... \n\n\n\n\n\nWhen demand is great...
1,1471,46423,"Los Angeles, CA",13-03-2020,"Which is your favorite Mad Max? The original, ..."
2,1472,46424,"Ithaca, NY",13-03-2020,Call your reps today &amp; demand they pass th...
3,1473,46425,"Vienna, Austria",13-03-2020,My partner: since I'm immuno compromised and c...
4,1474,46426,Chicago,13-03-2020,Empty shelves at the grocery store right now: ...
...,...,...,...,...,...
2324,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...
2325,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...
2326,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...
2327,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...


In [27]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re

def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # removing @ mention
    text = re.sub(r'#', '', text)  # removing # symbol
    text = re.sub(r'RT[\s]+', '', text)  # removing RT followed by space
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

# Define the dataset class for inference
class InferenceDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        tweet_text = str(self.data.loc[index, 'OriginalTweet'])
        inputs = self.tokenizer.encode_plus(
            tweet_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(inputs['token_type_ids'], dtype=torch.long),
        }

# Load the saved best model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest', ignore_mismatched_sizes=True)
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest', num_labels=5, ignore_mismatched_sizes=True)
model.load_state_dict(torch.load(r'C:\Users\USER\Downloads\best_roberta_sentiment_model.pth'))
model.to(device)
model.eval()

# Load the data for inference and clean the 'OriginalTweet' column
data_for_inference = data_for_inference.copy()  # Make a copy to avoid modifying the original DataFrame
data_for_inference['OriginalTweet'] = data_for_inference['OriginalTweet'].apply(cleanTxt)  # Apply text cleaning

# Print the column names
print(data_for_inference.columns)

# Create the inference dataset and data loader
inference_dataset = InferenceDataset(data_for_inference, tokenizer, MAX_LEN)
inference_loader = DataLoader(inference_dataset, batch_size=1)  # Batch size of 1 for inference

# Perform inference and create the "target" column
predicted_sentiments = []
with torch.no_grad():
    for batch in inference_loader:
        input_ids = batch['ids'].to(device)
        attention_mask = batch['mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        predicted_sentiments.extend(predicted_labels.cpu().numpy())

# Define the mapping from numeric labels to original sentiment labels
label_map = {0: 'Extremely Negative', 1: 'Positive', 2: 'Extremely Positive', 3: 'Negative', 4: 'Neutral'}

# Map the predicted numeric labels to their original sentiment labels
data_for_inference['Sentiment'] = predicted_sentiments
data_for_inference['Sentiment'] = data_for_inference['Sentiment'].map(label_map)

# Save the DataFrame with the original sentiment labels
data_for_inference.to_csv('output.csv', index=False)  # Change the file name accordingly


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')


In [30]:
data_for_inference

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1470,46422,"Ontario, Canada",13-03-2020,Hey idiots... When demand is greater than supp...,Negative
1,1471,46423,"Los Angeles, CA",13-03-2020,"Which is your favorite Mad Max? The original, ...",Positive
2,1472,46424,"Ithaca, NY",13-03-2020,Call your reps today &amp; demand they pass th...,Negative
3,1473,46425,"Vienna, Austria",13-03-2020,My partner: since I'm immuno compromised and c...,Extremely Negative
4,1474,46426,Chicago,13-03-2020,Empty shelves at the grocery store right now: ...,Negative
...,...,...,...,...,...,...
2324,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Extremely Positive
2325,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative
2326,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics was on talking about he...,Neutral
2327,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Negative


In [37]:
final_df = pd.concat([df,data_for_inference])

In [41]:
final_df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,2020-02-03 00:00:00,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",2020-02-03 00:00:00,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,2020-02-03 00:00:00,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,2020-02-03 00:00:00,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",2020-03-03 00:00:00,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...,...,...,...,...
2324,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Extremely Positive
2325,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative
2326,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics was on talking about he...,Neutral
2327,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Negative


In [48]:
%%time
from bertopic import BERTopic
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 7)
headline_topics, _ = model.fit_transform(final_df.OriginalTweet)

2024-03-25 13:04:38,284 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/119 [00:00<?, ?it/s]

2024-03-25 13:04:40,624 - BERTopic - Embedding - Completed ✓
2024-03-25 13:04:40,625 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-25 13:05:05,053 - BERTopic - Dimensionality - Completed ✓
2024-03-25 13:05:05,055 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-25 13:05:05,235 - BERTopic - Cluster - Completed ✓
2024-03-25 13:05:05,240 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-25 13:05:05,622 - BERTopic - Representation - Completed ✓


CPU times: user 41.9 s, sys: 510 ms, total: 42.4 s
Wall time: 27.6 s


In [50]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head(10)

Number of topics: 74


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1732,-1_the_to_and_of,"[the, to, and, of, in, is, for, coronavirus, c...","[If you are not feeling well, please donÂ’t go..."
1,0,211,0_panic_buying_food_people,"[panic, buying, food, people, 19, covid, the, ...",[Is it only my area that's running out of food...
2,1,177,1_coronapocalypse_grocery_coronavirus_store,"[coronapocalypse, grocery, coronavirus, store,...",[Scenes from grocery store battle ground part ...
3,2,133,2_stock_food_up_coronavirus,"[stock, food, up, coronavirus, on, you, to, an...",[Staying home due to the coronavirus? Here's w...
4,3,99,3_grocery_store_covid_19_supermarket,"[grocery, store, covid_19, supermarket, the, m...","[I don't usually do that kind of tweet, but I ..."
5,4,95,4_toilet_paper_toiletpaper_coronavirus,"[toilet, paper, toiletpaper, coronavirus, toil...",[People really out here working for toilet pap...
6,5,80,5_toilet_paper_covid_19_roll,"[toilet, paper, covid_19, roll, supermarket, g...",[Welp time to stock up on can food and yes.......
7,6,54,6_prices_ticket_flight_flights,"[prices, ticket, flight, flights, travel, airl...",[Airlines: Plane ticket prices are dropping be...
8,7,50,7_online_shopping_19_covid,"[online, shopping, 19, covid, use, spring, clo...",[Covid-19 could have a permanent effect on sho...
9,8,50,8_sick_employees_workers_store,"[sick, employees, workers, store, staff, paid,...","[Two different emails, @gamestop is providing ..."


In [51]:
a_topic = freq.iloc[1]["Topic"] # Select the 1st topic
model.get_topic(a_topic) # Show the words and their c-TF-IDF scores
model.visualize_barchart(top_n_topics=6)

In [52]:
# Select most 3 similar topics
similar_topics, similarity = model.find_topics("panic", top_n = 3)
similar_topics 

[16, 0, 60]

In [53]:
most_similar = similar_topics[0]
print("Most Similar Topic Info: \n{}".format(model.get_topic(most_similar)))
print("Similarity Score: {}".format(similarity[0]))

Most Similar Topic Info: 
[('panic', 0.0280350328879913), ('all', 0.016946227865205703), ('food', 0.01612713855153969), ('manila', 0.016060241701531062), ('there', 0.015414076766025424), ('of', 0.015157065604871699), ('buying', 0.014508528841070897), ('hysteria', 0.01308579888332456), ('we', 0.012445966468889945), ('amp', 0.012321839925398339)]
Similarity Score: 0.5829969048500061
